In [1]:
import numpy as np
from numpy import linalg as LA
import pandas as pd
import matplotlib.pyplot as plt
from scipy.constants import physical_constants as constants, angstrom

In [6]:
# Constants and Data
hbar = 1973.269804
inv_cm_to_eV = constants['inverse meter-electron volt relationship'][0]*100
eV_to_inv_cm = constants['electron volt-inverse meter relationship'][0]/100

molecular_data = pd.read_csv('data/molecular_data.csv', index_col='molecule')
molecular_data['mu'] *= 9.3149410372e8
# print(molecular_data)

In [ ]:
molecule = 'CO'
# Rotational Number
l = 20

molecular_parameters = (D_e, r_e, mu, alpha) = molecular_data.loc[molecule]

In [4]:
# Potential
def modified_morse(r):
    x = (r - r_e) / r
    potential = D_e * (np.exp(-2*alpha*x) - 2*np.exp(-alpha*x)) + ((l*(l+1) * hbar**2) / (2* mu * np.power(r,2)))
    return potential

In [5]:
step = 0.001
r0, rf = 0.6, 5-step

r_vals = np.arange(r0, rf, step)
N = len(r_vals)

In [6]:
# B-Matrix
B = np.identity(N, dtype=float) * 10
B += np.diag(np.ones(N-1), 1)
B += np.diag(np.ones(N-1), -1)
B /= 12

# A-Matrix
A = np.identity(N, dtype=float) * -2
A += np.diag(np.ones(N-1), 1)
A += np.diag(np.ones(N-1), -1)
A *= (-(hbar**2) / (2 * mu * step**2))

# Potential Matrix
V = np.identity(N, dtype=float)
np.fill_diagonal(V, modified_morse(r_vals))

# Hamiltonian Matrix
H = np.matmul(LA.inv(B), A) + V

In [7]:
E_vals, psi_vals = LA.eig(H)

# Keep only negative eigenvalues and corresponding eigenvectors
E_vals_pos = list(np.where(E_vals<0)[0])
E_vals = E_vals[E_vals_pos]
psi_vals = psi_vals[:,E_vals_pos]

# Sort in descending order (most negative eigenvalues to least negative)
sorted_E_vals_pos = np.argsort(E_vals)
E_vals = E_vals[sorted_E_vals_pos]
psi_vals = psi_vals[:,sorted_E_vals_pos]

In [8]:
# Eigenvalues data
eigenvalues = pd.Series(np.round(E_vals, 5), name=f'{molecule}')

# Eigenvalues are sorted
print(eigenvalues[:10])

0   -10.99255
1   -10.73378
2   -10.48298
3   -10.23994
4   -10.00440
5    -9.77614
6    -9.55494
7    -9.34059
8    -9.13286
9    -8.93156
Name: CO, dtype: float64


In [9]:
eigenvalues*eV_to_inv_cm

0     -88660.895009
1     -86573.774204
2     -84550.935784
3     -82590.685986
4     -80690.927767
           ...     
159    -1841.766958
160    -1403.969233
161     -963.025946
162     -519.017752
163      -71.944652
Name: CO, Length: 164, dtype: float64